# torch.nn           
torch.nn是专门为深度学习而设计的模块。torch.nn的核心数据结构是`Module`，它是一个抽象概念，既可以表示神经网络中的某个层（layer），也可以表示一个包含很多层的神经网络。   
在实际使用中，最常见的做法是继承`nn.Module`，撰写自己的网络/层。

## 1.1 自定义层              
如何用nn.Module实现自己的全连接层。全连接层，又名仿射层，输出$\textbf{y}$和输入$\textbf{x}$满足$\textbf{y=Wx+b}$，$\textbf{W}$和$\textbf{b}$是可学习的参数

In [1]:
import torch as t
from torch import nn

In [2]:
class Linear(nn.Module):
    def __init__(self, in_f, out_f):
        # 等价于 nn.Module.__init(self)
        super(Linear, self).__init__()
        self.w = nn.Parameter(t.randn(in_f, out_f))
        self.b = nn.Parameter(t.randn(out_f))
       
    def forward(self, x):
        x = x.mm(self.w)
        return x + self.b.expand_as(x)

In [3]:
layer = Linear(4, 3)
input = t.randn(2, 4)
output = layer(input)
output

tensor([[ 1.2630, -1.2677, -0.5461],
        [ 0.9991, -1.5613,  0.1438]], grad_fn=<AddBackward0>)

In [4]:
for name, parameter in layer.named_parameters():
    print(name, parameter)

w Parameter containing:
tensor([[ 0.2599,  0.9552,  0.3724],
        [ 1.1564,  1.9379,  0.6402],
        [-0.7190,  0.5199, -0.1499],
        [-0.3542,  0.5394,  0.5154]], requires_grad=True)
b Parameter containing:
tensor([1.6036, 0.0414, 0.0779], requires_grad=True)


### 在自定义层时，需注意以下几点：
(1) 自定义层`Linear`必须继承`nn.Module`，并且在其构造函数中需调用`nn.Module`的构造函数，即`super(Linear, self).__init__()` 或`nn.Module.__init__(self)`，推荐使用第一种用法，尽管第二种写法更直观。   


(2) 在构造函数`__init__`中必须自己定义可学习的参数，并封装成`Parameter`，如在本例中我们把`w`和`b`封装成`parameter`。   
- **注：**`parameter`是一种特殊的`Tensor`，但其默认需要求导（requires_grad = True），感兴趣的读者可以通过`nn.Parameter??`，查看`Parameter`类的源代码。
     
     
(3) `forward`函数实现前向传播过程，其输入可以是一个或多个tensor。    


(4) 无需写反向传播函数，nn.Module能够利用autograd自动实现反向传播，这点比Function简单许多。    


(5) 使用时，直观上可将layer看成数学概念中的函数，调用layer(input)即可得到input对应的结果。它等价于`layers.__call__(input)`，在`__call__`函数中，主要调用的是 `layer.forward(x)`，另外还对钩子做了一些处理。所以在实际使用中应尽量使用`layer(x)`而不是使用`layer.forward(x)`    


(6) `Module`中的可学习参数可以通过`named_parameters()`或者`parameters()`返回迭代器，前者会给每个parameter都附上名字，使其更具有辨识度。

可见利用Module实现的全连接层，比利用`Function`实现的更为简单，因其不再需要写反向传播函数

## 1.2 多层感知机      
Module能够自动检测到自己的`Parameter`，并将其作为学习参数。除了`parameter`之外，Module还包含子`Module`，主Module能够递归查找子`Module`中的`parameter`。    
下面再来看看稍微复杂一点的网络，多层感知机。

In [5]:
class Perceptron(nn.Module):
    def __init__(self, in_f, hidden_f, out_f):
        nn.Module.__init__(self)
        # 利用前面定义的全连接层
        self.layer1 = Linear(in_f, hidden_f)
        self.layer2 = Linear(hidden_f, out_f)
        
    def forward(self, x):
        x = self.layer1(x)
        x = t.sigmoid(x)
        return self.layer2(x)

In [6]:
perceptron = Perceptron(3, 4, 1)
input = t.randn(4, 3)
out = perceptron(input)
out

tensor([[-1.6494],
        [-1.2661],
        [-1.0140],
        [-1.1443]], grad_fn=<AddBackward0>)

In [7]:
for name, param in perceptron.named_parameters():
    print(name, param, param.size())

layer1.w Parameter containing:
tensor([[ 0.1484, -1.4652,  0.1285, -1.0985],
        [-0.7489, -0.4175, -0.0524, -1.5510],
        [ 0.6713, -1.7076,  0.6530, -0.6865]], requires_grad=True) torch.Size([3, 4])
layer1.b Parameter containing:
tensor([ 0.1396, -1.1325,  1.4509,  0.3764], requires_grad=True) torch.Size([4])
layer2.w Parameter containing:
tensor([[ 0.4581],
        [-0.4217],
        [-0.7926],
        [-0.8945]], requires_grad=True) torch.Size([4, 1])
layer2.b Parameter containing:
tensor([-0.0226], requires_grad=True) torch.Size([1])


可见，即使是稍复杂的多层感知机，其实现依旧很简单。 构造函数`__init__`中，可利用前面自定义的Linear层(module)，作为当前module对象的一个子module，它的可学习参数，也会成为当前module的可学习参数。

### module中parameter的命名规范：
- 对于类似`self.param_name = nn.Parameter(t.randn(3, 4))`，命名为`param_name`
- 对于子Module中的parameter，会其名字之前加上当前Module的名字。如对于`self.sub_module = SubModel()`，SubModel中有个parameter的名字叫做param_name，那么二者拼接而成的parameter name 就是`sub_module.param_name`。

### PS：查看关于layer文档  
为方便用户使用，PyTorch实现了神经网络中绝大多数的layer，这些layer都继承于nn.Module，封装了可学习参数`parameter`，并实现了forward函数，且很多都专门针对GPU运算进行了CuDNN优化，其速度和性能都十分优异。       

对nn.Module中的所有层的具体内容，读者可参照官方文档或在IPython/Jupyter中使用nn.layer?来查看。阅读文档时应主要关注以下几点：

- 构造函数的参数，如nn.Linear(in_features, out_features, bias)，需关注这三个参数的作用。
- 属性、可学习参数和子module。如nn.Linear中有`weight`和`bias`两个可学习参数，不包含子module。
- 输入输出的形状，如nn.linear的输入形状是(N, input_features)，输出为(N，output_features)，N是batch_size。

这些自定义layer对输入形状都有假设：输入的不是单个数据，而是一个batch。输入只有一个数据，则必须调用`tensor.unsqueeze(0)` 或 `tensor[None]`将数据伪装成batch_size=1的batch

## 1.3 nn.functional       
也是nn中常用的模块，nn中的大多数layer，在`functional`中都有一个与之相对应的函数。   

`nn.functional`中的函数和`nn.Module`的主要区别：   
- 用nn.Module实现的layers是一个特殊的类，都是由`class layer(nn.Module)`定义，会自动提取可学习的参数   
- `nn.functional`中的函数更像是纯函数，由`def function(input)`定义    

下面举例说明functional的使用，并指出二者的不同之处

In [7]:
input = t.randn(2, 3)
model = nn.Linear(3, 4)
output1 = model(input)
output2 = nn.functional.linear(input, model.weight, model.bias)
output1, output2

(tensor([[-0.4439, -0.2176,  0.3737, -0.9774],
         [-0.5911, -0.1468,  0.5085, -0.7642]], grad_fn=<AddmmBackward>),
 tensor([[-0.4439, -0.2176,  0.3737, -0.9774],
         [-0.5911, -0.1468,  0.5085, -0.7642]], grad_fn=<AddmmBackward>))

### 如何选择`nn.Module`和`nn.functional`     
模型有可学习的参数，最好用nn.Module，否则既可以使用nn.functional也可以使用nn.Module，二者在性能上没有太大差异，具体的使用取决于个人的喜好。      

如激活函数（ReLU、sigmoid、tanh），池化（MaxPool）等层由于没有可学习参数，则可以使用对应的functional函数代替，而对于卷积、全连接等具有可学习参数的网络建议使用nn.Module    

**Dropout**    
虽然dropout操作也没有可学习操作，但建议还是使用`nn.Dropout`而不是`nn.functional.dropout`，因为dropout在训练和测试两个阶段的行为有所差别，使用`nn.Module`对象能够通过`model.eval`操作加以区分

In [8]:
from torch.nn import functional as F

class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.pool(F.relu(self.conv1(x)), 2)
        x = F.pool(F.relu(self.conv2(x)), 2)
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

        

对于不具备可学习参数的层（激活层、池化层等），将它们用函数代替，这样则可以不用放置在构造函数`__init__`中。   

对于有可学习参数的模块，也可以用functional来代替，只不过实现起来较为繁琐，需要手动定义参数parameter，如前面实现自定义的全连接层，就可将weight和bias两个参数单独拿出来，在构造函数中初始化为parameter。

In [12]:
class myLinear(nn.Module):
    def __init__(self):
        super(myLinear, self).__init__()
        self.w = nn.Parameter(t.randn(3, 4))
        self.b = nn.Parameter(t.zeros(3))
        
    def forward(self, input):
        return F.Linear(input, w, b)

## 1.4 `nn.Module` 深入理解     
nn.Module 基类的构造函数：    
```python  
    def __init__(self):
        self._backend = thnn_backend
        self._parameters = OrderedDict()
        self._buffers = OrderedDict()
        self._backward_hooks = OrderedDict()
        self._forward_hooks = OrderedDict()
        self._forward_pre_hooks = OrderedDict()
        self._state_dict_hooks = OrderedDict()
        self._load_state_dict_pre_hooks = OrderedDict()
        self._modules = OrderedDict()
        self.training = True
```   
  
部分参数解释：    

- `_parameters`：字典，保存用户直接设置的parameter，`self.param1 = nn.Parameter(t.randn(3, 3))`会被检测到，在字典中加入一个key为'param'，value为对应parameter的item。而self.submodule = nn.Linear(3, 4)中的parameter则不会存于此。
- `_modules`：子module，通过`self.submodel = nn.Linear(3, 4)`指定的子module会保存于此。
- `_buffers`：缓存。如batchnorm使用momentum机制，每次前向传播需用到上一次前向传播的结果。
- `_backward_hooks`与`_forward_hooks`：钩子技术，用来提取中间变量，类似variable的hook。
- `training`：BatchNorm与Dropout层在训练阶段和测试阶段中采取的策略不同，通过判断training值来决定前向传播策略。

上述几个属性中，`_parameters`、`_modules`和`_buffers`这三个字典中的键值，都可以通过`self.key`方式获得，效果等价于`self._parameters['key']`

In [13]:
nn.Module??

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.param1 = nn.Parameter(t.randn(3, 3))
        self.submodule1 = nn.Linear(3, 4)
        
    def forward(self, x):
        x = self.param1.mm(x)
        x = self.submodule1(x)
        return x
    
net = Net()
net

Net(
  (submodule1): Linear(in_features=3, out_features=4, bias=True)
)

In [3]:
net._modules

OrderedDict([('submodule1', Linear(in_features=3, out_features=4, bias=True))])

In [4]:
net._parameters

OrderedDict([('param1', Parameter containing:
              tensor([[-1.3981,  0.8565, -0.7410],
                      [-0.0970,  1.5115,  0.5026],
                      [-0.2168,  0.5387, -2.2013]], requires_grad=True))])

### (1) 子module    
nn.Module在实际使用中可能层层嵌套，为方便用户访问各个子module，nn.Module实现了很多方法：   
- 函数`children`可以查看直接子module   
- 函数`module`可以查看所有的子module（包括当前module）   

与之相对应的还有函数`named_childen`和`named_modules`，其能够在返回module列表的同时返回它们的名字。

In [5]:
for i in net.children():
    print(i)

Linear(in_features=3, out_features=4, bias=True)


### (2) training 属性     
对于batchnorm、dropout、instancenorm等在训练和测试阶段行为差距巨大的层，如果在测试时不将其training值设为True，则可能会有很大影响，这在实际使用中要千万注意。            

虽然可通过直接设置`training`属性，来将子module设为train和eval模式，但这种方式较为繁琐，因如果一个模型具有多个dropout层，就需要为每个dropout层指定training属性。     
更为推荐的做法是调用`model.train()`函数，它会将当前module及其子module中的所有training属性都设为True，相应的，`model.eval()`函数会把training属性都设为False

In [6]:
print(net.training, net.submodule1.training)
net.eval()
print(net.training, net.submodule1.training)

True True
False False


### (3) 钩子函数    
`register_forward_hook`与`register_backward_hook`      
这两个函数的功能类似于variable函数的`register_hook`，可在module前向传播或反向传播时注册钩子。   
每次前向传播执行结束后会执行钩子函数（hook）。
- 前向传播的钩子函数具有如下形式：`hook(module, input, output) -> None`   
- 反向传播则具有如下形式：`hook(module, grad_input, grad_output) -> Tensor or None`。    
  
钩子函数不应修改输入和输出，并且在使用后应及时删除，以避免每次都运行钩子增加运行负载。    
钩子函数主要用在获取某些中间结果的情景，如中间某一层的输出或某一层的梯度。这些结果本应写在forward函数中，但如果在forward函数中专门加上这些处理，可能会使处理逻辑比较复杂，这时候使用钩子技术就更合适一些。     

下面考虑一种场景，有一个预训练好的模型，需要提取模型的某一层（不是最后一层）的输出作为特征进行分类，但又不希望修改其原有的模型定义文件，这时就可以利用钩子函数。下面给出实现的伪代码。   

```python
model = VGG()
features = t.Tensor()
def hook(module, input, output):
    '''把这层的输出拷贝到features中'''
    features.copy_(output.data)
    
handle = model.layer8.register_forward_hook(hook)
_ = model(input)

# 用完hook后删除    
handle.remove()
```

### (4) 魔法方法  
`nn.Module`对象在构造函数中的行为看起来有些怪异，如果想要真正掌握其原理，就需要看两个魔法方法`__getattr__`和`__setattr__`。    

在Python中有两个常用的buildin方法`getattr`和`setattr`  
- `getattr(obj, 'attr1')`等价于`obj.attr`     
如果`getattr`函数无法找到所需属性，Python会转而调用`obj.__getattr__('attr1')`方法，即`getattr`函数无法找到的交给`__getattr__`函数处理，没有实现`__getattr__`或者`__getattr__`也无法处理的就会raise AttributeError        
- `setattr(obj, 'name', value)`等价于`obj.name=value`     
如果obj对象实现了`__setattr__`方法，setattr会直接调用`obj.__setattr__('name', value)`，否则调用buildin方法。      
  
总结一下：  
- result  = obj.name会调用buildin函数`getattr(obj, 'name')`，如果该属性找不到，会调用`obj.__getattr__('name')`
- obj.name = value会调用buildin函数`setattr(obj, 'name', value)`，如果obj对象实现了`__setattr__`方法，`setattr`会直接调用`obj.__setattr__('name', value')`

nn.Module实现了自定义的`__setattr__`函数     
当执行`module.name=value`时，会在`__setattr__`中判断value是否为`Parameter`或`nn.Module`对象
- 如果是则将这些对象加到`_parameters`和`_modules`两个字典中
- 而如果是其它类型的对象，如`Variable`、`list`、`dict`等，则调用默认的操作，将这个值保存在`__dict__`中

In [7]:
module = nn.Module()
module.param = nn.Parameter(t.ones(2, 2))
module._parameters

OrderedDict([('param', Parameter containing:
              tensor([[1., 1.],
                      [1., 1.]], requires_grad=True))])

In [16]:
submd1 = nn.Linear(2, 2)
submd2 = nn.Linear(2, 2)
sub_mds = [submd1, submd2]
module.submodules = sub_mds

# 对于list对象，调用buildin函数，保存在__dict__中
print('modules', module._modules)
print("__dict__['submodules']:", module.__dict__.get('submodules'))

modules OrderedDict()
__dict__['submodules']: [Linear(in_features=2, out_features=2, bias=True), Linear(in_features=2, out_features=2, bias=True)]


In [19]:
module_list = nn.ModuleList(sub_mds)
module.submodules = module_list 

print('ModuleList is instance of nn.Module: ---', isinstance(module_list, nn.Module))
print('modules', module._modules)
print("__dict__['submodules']:", module.__dict__.get('submodules'))

ModuleList is instance of nn.Module: --- True
modules OrderedDict([('submodules', ModuleList(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
))])
__dict__['submodules']: None


`_modules`和`_parameters`中的item未保存在`__dict__`中，所以默认的getattr方法无法获取它    
因而`nn.Module`实现了自定义的`__getattr__`方法，如果默认的`getattr`无法处理，就调用自定义的`__getattr__`方法，尝试从`_modules`、`_parameters`和`_buffers`这三个字典中获取。

In [22]:
getattr(module, 'training')

True

### Optional 1: save model   
在PyTorch中保存模型十分简单，所有的Module对象都具有state_dict()函数，返回当前Module所有的状态数据。   

将这些状态数据保存后，下次使用模型时即可利用`model.load_state_dict()`函数将状态加载进来。  
优化器（optimizer）也有类似的机制，不过一般并不需要保存优化器的运行状态

In [23]:
t.save(net.state_dict(), 'net.pth')

net2 = Net()
net2.load_state_dict(t.load('net.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## 1.5  nn和autograd的关系
nn.Module利用的也是autograd技术，其主要工作是实现前向传播。在forward函数中，nn.Module对输入的tensor进行的各种操作，本质上都是用到了autograd技术。这里需要对比autograd.Function和nn.Module之间的区别：
- autograd.Function利用了Tensor对autograd技术的扩展，为autograd实现了新的运算op，不仅要实现前向传播还要手动实现反向传播
- nn.Module利用了autograd技术，对nn的功能进行扩展，实现了深度学习中更多的层。只需实现前向传播功能，autograd即会自动实现反向传播
- nn.functional是一些autograd操作的集合，是经过封装的函数

作为两大类扩充PyTorch接口的方法，我们在实际使用中应该如何选择呢？    
- 如果某一个操作，在autograd中尚未支持，那么只能实现Function接口对应的前向传播和反向传播
- 如果某些时候利用autograd接口比较复杂，则可以利用Function将多个操作聚合，实现优化，正如第三章所实现的`Sigmoid`一样，比直接利用autograd低级别的操作要快  
- 而如果只是想在深度学习中增加某一层，使用nn.Module进行封装则更为简单高效。